<a href="https://colab.research.google.com/github/yufang934/deeplearning/blob/main/transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##遷移學習+mlp

使用別人訓練好的模型來用

In [ ]:
import zipfile
f = zipfile.ZipFile("drive/My Drive/train.zip")
#解壓縮
f.extractall()

In [ ]:
import glob
#glob:可以把所有相符的檔案叫出來(https://www.itread01.com/articles/1476166832.html)
import pandas as pd
dogs = glob.glob("train/dog.*")
cats = glob.glob("train/cat.*")
df = pd.DataFrame({
    "path":dogs+cats,
    "target":[0] * len(dogs) + [1] * len(cats)
    #list[0]
})
print(df)
print(len(dogs))
print(len(cats))

In [ ]:
import random
from tensorflow.keras.preprocessing.image import load_img
idx = random.randint(0,24999)
#把size改成跟借來的模型依樣
load_img(df["path"][idx], target_size=(224,224))

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
#真的要開始切分和訓練的時候，請你把你的pandas轉成numpy的array
x, y = np.array(df["path"]), np.array(df["target"])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
print(x_train.shape)
print(y_test.shape)
print(x_train.shape)
print(y_test.shape)

In [ ]:
from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
#dropout隨機的斷開一些神經元
from tensorflow.keras.layers import BatchNormalization
#使用vgg16模型
#include_top:當初訓練好的腦袋
cnn = vgg16.VGG16(include_top=False, input_shape=(224, 224, 3))
#CNN(眼睛)
for l in cnn.layers:
  l.trainable = False
mlp = [
    BatchNormalization(),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(2, activation="softmax")
]
model = Sequential(cnn.layers + mlp)
model.summary()

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
model.compile(loss=SparseCategoricalCrossentropy(),
       #小彈珠優化法
       optimizer=Adam(),
       metrics=["accuracy"])

In [ ]:
#讀圖後回傳
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py
import numpy as np

def getdata(x, y, batch=20):
    idx = np.random.randint(0, x.shape[0], size=batch)
    orix, batchx, batchy = [], [], y[idx]
    for p in x[idx]:
        img = load_img(p, target_size=(224, 224))
        img_np = np.array(img)
        orix.append(img_np)
        img_pre = vgg16.preprocess_input(img_np)
        batchx.append(img_pre)
    return (np.array(orix), np.array(batchx), np.array(batchy))

testori, testx, testy = getdata(x_train, y_train, 20)
print(testori.shape)
print(testx.shape)
print(testy.shape)

In [ ]:
testimg, testx, testy = getdata(x_test, y_test, 200)

for i in range(20):
    print("-" * 15, i, "-" * 15)
    trainimg, trainx, trainy = getdata(x_train, y_train, 20)
    result = model.train_on_batch(trainx, trainy)
    print("Train:", result)
    result = model.test_on_batch(testx, testy)
    print("Validate:", result)

In [ ]:
# model.predict(trainx)
testimg, testx, testy = getdata(x_test, y_test, 1000)
model.evaluate(testx, testy)

In [ ]:
# subplot(總高度, 總寬度, ith)
import numpy as np
import matplotlib.pyplot as plt

testimg, testx, testy = getdata(x_test, y_test, 1000)
pre = model.predict_classes(testx)
trans = ["dog", "cat"]

idx = np.nonzero(pre != testy)[0]
idx = idx[:200]
false_img = testimg[idx]
false_label = testy[idx]
false_pre = pre[idx]

plt.figure(figsize=(14, 42))
width = 10
height = len(false_img) // width + 1
for i in range(len(false_img)):
    plt.subplot(height, width, i+1)
    title = "[O]:{}\n[P]:{}".format(trans[false_label[i]], trans[false_pre[i]])
    plt.title(title)
    plt.axis("off")
    plt.imshow(false_img[i])

In [ ]:
# pip install pillow
import PIL
import requests
url = input("輸入網址:")
h = {
    "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
}
response = requests.get(url, stream=True, verify=False, headers=h)
img = PIL.Image.open(response.raw).resize((224, 224))
img_np = np.array(img)
test = vgg16.preprocess_input(img_np.reshape(1, 224, 224, 3))
probs = model.predict(test)[0]
for i, p in enumerate(probs):
    print(trans[i], "的機率是:", round(p, 3))
ans = model.predict_classes(test)[0]
print("應該是:", trans[ans])
plt.imshow(img_np)